# DM3 Translator
* This file is used to translate the raw DM3 data file into standard H5 file. 

* The data will be initially processed and stored in the same file for future analysis. 
 - The process includes: ZLP alignment, Thickness computing, ZLP substraction. 
 - Please be aware that the data is limited to low-loss data so far. <br>
 - If the raw data is core-loss signal, please include additional low-loss data for the ZLP alignment.

# Import Software:
* Standard distribution (include numpy, scipy, matplotlib, sci-kit learn).
* **hyperspy** : can load and process DM3 data directly. (import hyperspy.api, not hyperspy)
* **pycroscopy** : mainly used here for plotting purposes only. 
* **hyperspy_tools** : to extract EELS zlp.

In [1]:
#Import packages

# Ensure that this code works on both python 2 and python 3
from __future__ import division, print_function, absolute_import, unicode_literals

# basic numeric computation:
import numpy as np

# basic system operation:
import os

# The package used for creating and manipulating HDF5 files:
import h5py
import hyperspy.api as hs

# To extract EELS ZLP, put 'eels.py' from hyperspy_tools in the python path and then import here:
from eels import extract_ZLP

# Plotting and visualization:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# multivariate analysis:
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF

# finally import pycroscopy:
import pycroscopy as px

# Load DM3 data:
* Use **hyperspy** to read DM3 data.
* Reshape the data as h5 file required.
## Tips:
* The 'np.reshape' will reshape the matrix acoording to the C memory order, i.e. the most rapidly changing index comes last: (k,j,i)
* The h5 file for pycroscopy requires the 3D EELS data (i,j,k) to be flattened as 2D (i*j, k) in the C memory order as well, i.e. the order of row in the flattened data is [0,0],[0,1]...[0,N-1],[1,0],[1,1] ...

In [2]:
folder_path=''
file_name='EELS Spectrum Image.dm3'
raw = hs.load(folder_path+file_name,signal_type='EELS')
data = raw.data
np.shape(data)
raw.metadata

(18, 19, 1340)

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EELS
│       │       ├── collection_angle = 50.0
│       │       └── frame_number = 1
│       ├── acquisition_mode = STEM
│       ├── beam_current = 0.0
│       ├── beam_energy = 80.0
│       ├── camera_length = 400.0
│       ├── convergence_angle = 20.0
│       ├── magnification = 2500000.0
│       └── microscope = JEOL COM
├── General
│   ├── date = 2016-05-07
│   ├── original_filename = EELS Spectrum Image.dm3
│   ├── time = 21:32:29
│   └── title = EELS Spectrum Image2
└── Signal
    ├── Noise_properties
    │   └── Variance_linear_model
    │       ├── gain_factor = 0.20789135992527008
    │       └── gain_offset = 0.0
    ├── binned = True
    ├── quantity = Electrons (Counts)
    └── signal_type = EELS

In [3]:
count = 0
data_shape = np.shape(data)
ind_mat = np.zeros(data_shape)
for (ind1,ind2,ind3), val in np.ndenumerate(data):
    ind_mat[ind1,ind2,ind3] = count
    count += 1
#print(np.shape(data))

h5_data = np.reshape(data,(-1,data_shape[2]))
h5_ind = np.reshape(ind_mat,(-1,data_shape[2]))
##present the ordering:
#make a function to transfer count of the memory order to 3D position.
#def getindex(number):
#    id1 = number//(data_shape[2]*data_shape[1])
#    res1 = number%(data_shape[2]*data_shape[1])
#    id2 = res1//data_shape[2]
#    id3 = res1 - id2*data_shape[2]
#    return (id1,id2,id3)

#for i in range(30):
#    print(getindex(h5_ind[i,0]))


# Prepare Ancillary Data and Translate into H5 file:
* Create the new file path.
* Specify the spectroscopic axis: minimum of the axis, and dispersion
* Translate the data into h5 file using **NumpyTranslator**

In [4]:
h5_path = os.path.join(folder_path, file_name[:-4] + '.h5')
print('The h5 file will be saved in the following path:')
print(h5_path)

image_cal = raw.original_metadata.ImageList.TagGroup0.ImageData.Calibrations.Dimension
pos_cal = image_cal.TagGroup0
spec_cal = image_cal.TagGroup2
print(image_cal)

# Generate the x / spectroscopic axis:
pos_pixel_size = pos_cal.Scale
pos_units = pos_cal.Units


spec_pixel_size = spec_cal.Scale
spec_min = (0-spec_cal.Origin)*spec_cal.Scale
spec_units = spec_cal.Units
spec_vec = np.linspace(spec_min, spec_min+spec_pixel_size*(data_shape[2]-1), data_shape[2])


# Generate attributes dictionary:
parm_dict=dict()
tem = raw.metadata.Acquisition_instrument.TEM
parm_dict['beam_energy'] = tem.beam_energy
parm_dict['convergence_angle'] = tem.convergence_angle
parm_dict['EELS_collection_angle']=tem.Detector.EELS.collection_angle
parm_dict['num_rows']=data_shape[0]
parm_dict['num_cols']=data_shape[1]
parm_dict['spec_length']=data_shape[2]
parm_dict['spec_axis']=spec_vec
parm_dict['spec_pixel_size']=spec_pixel_size
parm_dict['spec_unit']=spec_units
parm_dict['spatial_pixel_size']=pos_pixel_size
parm_dict['spatial_unit']=pos_units
for key in parm_dict:
    print(key, ':', parm_dict[key])

The h5 file will be saved in the following path:
EELS Spectrum Image.h5
├── TagGroup0
│   ├── Origin = 0.0
│   ├── Scale = 0.002550000324845314
│   └── Units = µm
├── TagGroup1
│   ├── Origin = 0.0
│   ├── Scale = 0.002550000324845314
│   └── Units = µm
└── TagGroup2
    ├── Origin = 100.0
    ├── Scale = 0.05000000074505806
    └── Units = eV

beam_energy : 80.0
convergence_angle : 20.0
EELS_collection_angle : 50.0
num_rows : 18
num_cols : 19
spec_length : 1340
spec_axis : [ -5.00000007  -4.95000007  -4.90000007 ...,  61.85000092  61.90000092
  61.95000092]
spec_pixel_size : 0.05000000074505806
spec_unit : eV
spatial_pixel_size : 0.002550000324845314
spatial_unit : µm


In [5]:
tran = px.io.NumpyTranslator()
h5_path = tran.translate(h5_path, h5_data, data_shape[0], data_shape[1], 
                         qty_name='Count', data_unit='', spec_name='Energy Loss',
                         spec_unit='eV', spec_val=spec_vec, spatial_unit='um', 
                         data_type='STS',translator_name='ASC', parms_dict=parm_dict)